# Project01 for L2 - Informatique

*In this project, you will be cleaning, analysing and visualising datasets recording stock prices.*

__**Date and time of submission: Tuesday 2023-04-18 at 20:00 in the evening. There is a 40% penalty for missing the deadline.**__

The commands to load the datasets into Pandas dataframes have been written for you, but you will still need to do all the other steps.

This notebook is written in a step-by-step manner, but you are free to modify it to suit your needs.

Note that there are three different sets of data, each of which is fairly large.

You are **strongly** encouraged to work on a small subset of the datasets first, and once you are satisfied that everything is working well, load a larger portion of the datasets, and eventually work on the entire datasets.

Another thing to bear in mind is that the three datasets are **very similar**: they have the same columns and the same number of rows. However, the values in the data sets are different, although they are **correlated**, as will be explained.


*Execute the following cell, you do not need to understand it, it just loads the appropriate packages so that you can work with the dataset*

In [ ]:
%pip install seaborn
%pip install pandas-datareader

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print("Data file found:", os.path.join(dirname, filename))

import numpy as np
import pandas as pd
import pandas_datareader.data as web
from pandas import Series, DataFrame
from random import Random
import timeit
import pickle

import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime, timedelta, date
import math

%matplotlib inline

# Loading the dataset

In the next cell you can configure what percentage of the data set to load (1%, 10%, 100%). Start with a small amount, e.g. 1%, to test your solutions and then gradually increase it (10%, 100%) to test larger parts of the dataset.

*Remember that you will need to come back here and re-execute all the cell when you want to change the percentage of the datasets used*

In [ ]:
# Expressed as a percentage (1, 10, or 100)
dataset_percentage = 1
# dataset_percentage = 10
# dataset_percentage = 100

This will actually load all three datasets, for the stocks named Apple, Pear and Orange, into the Pandas data frames named `dfapple`, `dfpear` and `dforange` correspondingly.

In [ ]:
dfapple = pd.read_pickle(f"data/Apple_{dataset_percentage}pct.pickle.gz")
dfpear = pd.read_pickle(f"data/Pear_{dataset_percentage}pct.pickle.gz")
dforange = pd.read_pickle(f"data/Orange_{dataset_percentage}pct.pickle.gz")

Here we print the descriptions of each of these data frames.

In [ ]:
display("dfapple:", dfapple.describe())
display("dfpear:", dfpear.describe())
display("dforange:", dforange.describe())

Finally, here we print the `head()` of the `dfapple` data frame. You can do the same for the rest of the dataframes if you like.

In [ ]:
dfapple.head()

# Project Assignment 1

The data in each of the data frames is corrupted. You need to pre-process (or clean) the data, making sure that a number of conditions are all met after the data has been processed in order for it to be considered "clean".

Start working on `dfapple` and once you have implemented the cleaning/pre-processing, do the same for the other two dataframes.

## Conditions to meet

### Condition 1 - Negative values
Data quality issue: Some records in the dataset have **negative** values that do not make sense. Specifically, the following columns are affected:

* `open`
* `close`
* `high`
* `low`
* `volume`

These columns must always have values that are **never negative**.

You need to *filter* the data set and *remove all rows* where *any* of the preceding columns have a negative value.

Put another way, you can *filter* the data set and *keep only the rows* where *none* of the preceding columns has a negative value.

### Condition 2 - abnormally high values
Data quality issue: some records in the data set have **abnormally high** positive values that make no sense. Specifically, the following columns are affected:

* `open`
* `close`
* `high`
* `low`

These columns must always have values that are **never abnormally high**.

*Abnormally high is interpreted to mean more than 100 times the average value of the column. For example, if the average of the `open` column is `10`, then any value above `100*10=1000` should be considered abnormally high.

You need to *filter* the data set and *remove* all the rows where *any* of the previous columns has an *abnormally high* value.

In other words, you can *filter* the data set and *keep only the rows* where *none* of the preceding columns has an *abnormally high* value.

### Condition 3 - Zero values

Data quality issue: Some records in the dataset have **zero values** which do not make sense. Specifically, the following columns are affected

* `open`
* `close`
* `high`
* `low`

These columns must always have values that are *never zero*.

You need to *filter* the data set and *remove all rows* where *any* of the preceding columns have a value of zero.

Put another way, you can *filter* the data set and *keep only the rows* where *none* of the preceding columns has a value of zero.

### Condition 4 - High/Low Values

Data quality issue: some records in the data set have **`high`** or **`low`** values that do not make sense. Specifically, the following columns are affected

* `high`
* `low`

These columns must always have values that are **always greater than or equal to each other**. More specifically, *in all rows*, the value of the column `high` **greater than or equal** to the value of the `low` column.

You need to *filter* the data set and *remove* all the rows where the value of the high column is less than the value of the low column.

In other words, you can *filter* the data set and *keep only the rows* where the value of the column `high` is *greater than or equal* to the value of the column `low`.

### Condition 5 - Open/Close values

Data quality problem: Some entries in the dataset have **`open` or `close` values**, which do not make sense. More specifically, the following columns are concerned:

* `open`
* `close`

These columns must always have values that are **inside of the range of the corresponding \[`high` -- `low`\] values**. More specifically, *in all rows*, the value of *both* columns `open` and `close` **must be larger or equal** to the value of the `low` column *and* **must be smaller or equal** to the value of the `high` column.

For example, it is abnormal if a row has the following values: `low=10, high=15, open=13, close=16`. In this case, the `close` value is larger than the value of `high`.

You must *filter* the dataset and *remove all the rows* where the values of the columns `open` or `close` is outside the `low`--`high` range values.

Said another way, you can *filter* the dataset and *keep only the rows* where the values of the columns `open` or `close` is *inside* the `low`--`high` range values.

### Condition 6 - Temporal order in rows
Data quality problem: Some entries in the dataset are **in the wrong temporal order**, which does not make sense. More specifically, the following column is concerned:

* `timestamp`

This column must always have values that are **correct chronological order**. 
For example, if in one row the value of `timestamp` is `2017-01-10 13:9:00` and in the **next** row the value of `timestamp` is `2017-01-10 13:2:00`, then the last row is not in correct chronological order and you must remove it

You must *filter* the dataset and *remove all the rows* where the value of the column `timestamp` in one row is **before** the value of the column `timestamp` in the **previous** row.

Said another way, you can *filter* the dataset and *keep only the rows* where the value of the column `timestamp` in one row is **after** the value of the column `timestamp` in the **previous** row.


#### Warm up
Before starting with cleaning up the data, it will be useful if you have a small piece of code that prints the contents of a column that you are working on, to be able to visualise the changes in the data.

For example: this is the visualisation of the `open` column in `dfapple` when 10% of the dataset is loaded:

![](Apple_open.png)

*Hint:* pass the parameter `ci=None` to the `sns.lineplot()` to significantly speed-up the drawing of the diagram.

Your code should follow. At the end of your code, you should have `clean` data in the `dfapple`, `dfpear` and `dforange` data frames.

In the following cells, use as many cells as you need (i.e., you can create new ones if you want).

# Project assignment 2

After the clean-up of the data, your task is to do a very **simple price prediction model**.

The values in all the columns in the three data sets are somehow related (but the exact way is unknown to you), so you need to investigate and try to find this relationship.

Start as before with a part of the dataset `dfapple` and try to see the relationship between the values of one of the columns, e.g. `open`, in the `dfpear` dataset.

You already know that the relationship is not exact (i.e. that there is some variability, which you can consider a tolerable prediction error).

You also know that the relationship is time-invariant. That is, it does **not change over time**.

The aim is that if you are given a values from one column from the `dfapple` dataset, you should be able to predict what the equivalent values of the `dfpear` and `dforange` values are.

As a more specific example, imagine that:

* at `2017-01-09 13:02:00` the value of `dforange` in the `open` column is 12 (dollars).
* at the same moment in time in the value of `dfpear` in the `open` column is 10 (dollars).
* Now, if the value of `dforange` in the `open` column at the next instant, e.g. at `2017-01-09 13:03:00`, the value is 13 (dollars):
  * **try to predict what the value of `dfpear` in the `open` column will be.**

*Hint:* Use the visualisation techniques to verify your predictions.
*Hint:* Calculate the difference between your predicted value and the actual value (which is available) to see how well you are doing.

The assessment for this exercise will also have a section at the end where a new set of data - unknown to you - will be provided and you will have to make the same prediction during the exam. To succeed in this task, you must make your prediction method **as general as possible**.

The only information you have is that the overall relationship will be **of the same type**, but not necessarily of the same magnitude.

## Explicit aims:

1. Given values in `dfapple` of the columns `open`,`close`, `high`, `low` predict the value of the same column in `dfpear` **and** `dforange` at the same point in time.
2. Given values in `dfapple` of the columns `open`,`close`, `high`, `low` and an unknown new dataset in a dataframe named `dfunknown`predict the value of the same column in `dfunknown` at the same point in time.
  

In the following cells, use as many cells as you need (i.e., you can create noew ones if you want).
